In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [12]:
df = pd.read_csv('./pokemon_alopez247.csv')

In [13]:
df = df[['isLegendary','Generation', 'Type_1', 'Type_2', 'HP', 'Attack', 'Defense', 'Sp_Atk', 'Sp_Def', 'Speed','Color','Egg_Group_1','Height_m','Weight_kg','Body_Style']]

In [14]:
df['isLegendary'] = df['isLegendary'].astype(int)

In [15]:
def dummy(df, categories):
    for i in categories:
        df_dummy = pd.get_dummies(df[i])
        df = pd.concat([df, df_dummy], axis=1)
        df = df.drop(i, axis=1)
    return df

In [16]:
df = dummy(df, ['Egg_Group_1', 'Body_Style', 'Color','Type_1', 'Type_2'])
df.head(5)

,isLegendary,Generation,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Height_m,Weight_kg,...,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,0,1,45,49,49,65,65,45,0.71,6.9,...,0,0,0,0,0,1,0,0,0,0
1,0,1,60,62,63,80,80,60,0.99,13.0,...,0,0,0,0,0,1,0,0,0,0
2,0,1,80,82,83,100,100,80,2.01,100.0,...,0,0,0,0,0,1,0,0,0,0
3,0,1,39,52,43,60,50,65,0.61,8.5,...,0,0,0,0,0,0,0,0,0,0
4,0,1,58,64,58,80,65,80,1.09,19.0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
def train_splitter(df, column): 
    df_train = df.loc[df[column] != 1]
    df_test = df.loc[df[column] == 1]
    
    df_train = df_train.drop(column, axis=1)
    df_test = df_test.drop(column, axis=1)

    return(df_train, df_test)

In [18]:
df_train, df_test = train_splitter(df, 'Generation')

In [19]:
def label_delineator(df_train, df_test, label):
    train_data = df_train.drop(label, axis=1).values
    train_labels = df_train[label].values
    test_data = df_test.drop(label, axis=1).values
    test_labels = df_test[label].values
    return(train_data, train_labels, test_data, test_labels)

In [20]:
train_data, train_labels, test_data, test_labels = label_delineator(df_train, df_test, 'isLegendary')

In [27]:
def normalizer_data(train_data, test_data):
    train_data = preprocessing.MinMaxScaler().fit_transform(train_data)
    test_data = preprocessing.MinMaxScaler().fit_transform(test_data)    
    return(train_data, test_data)

In [29]:
train_data, test_data = normalizer_data(train_data, test_data)

In [37]:
length = train_data.shape[1]
model = keras.Sequential()
model.add(keras.layers.Dense(500, activation='relu', input_shape=[length,]))
model.add(keras.layers.Dense(2, activation='softmax'))

In [38]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [39]:
model.fit(train_data, train_labels, epochs=400)

Epoch 1/400
18/18 [==============================] - 0s 2ms/step - loss: 0.6511 - accuracy: 0.6544
Epoch 2/400
18/18 [==============================] - 0s 2ms/step - loss: 0.4778 - accuracy: 0.9263
Epoch 3/400
18/18 [==============================] - 0s 2ms/step - loss: 0.3922 - accuracy: 0.9263
Epoch 4/400
18/18 [==============================] - 0s 2ms/step - loss: 0.3454 - accuracy: 0.9263
Epoch 5/400
18/18 [==============================] - 0s 2ms/step - loss: 0.3177 - accuracy: 0.9263
Epoch 6/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2998 - accuracy: 0.9263
Epoch 7/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2871 - accuracy: 0.9263
Epoch 8/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2777 - accuracy: 0.9263
Epoch 9/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2703 - accuracy: 0.9263
Epoch 10/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2640 - accuracy: 0.9263
Epoch 11/

In [42]:
loss_value, accuracy_value = model.evaluate(test_data, test_labels)
print(f'Our test accuracy was {accuracy_value}')

5/5 [==============================] - 0s 2ms/step - loss: 0.0355 - accuracy: 0.9801
Our test accuracy was 0.9801324605941772


In [66]:
def predictor(test_data, test_labels, index):
    prediction = model.predict(test_data)
    if np.argmax(prediction[index]) == test_labels[index]:
        print(f'This was correctly predicted to be a \"{test_labels[index]}\"!')
        return(prediction[index])
    else:
        print(f'This was incorrectly predicted to be a \"{np.argmax(prediction[index])}\". It was actually a \"{test_labels[index]}\".')
        return(prediction[index])

In [74]:
df['isLegendary'][142]

0

In [71]:
predictor(test_data, test_labels, 142)

This was correctly predicted to be a "0"!


array([0.99745613, 0.00254388], dtype=float32)